#### Notebook for image preprocessing and obtaining coarse localized AC-PC landmarks.

##### Steps
* Grab the raw NIfTI volumes.
* Perform isotropic resampling, cropping, bed removal, skull stripping.
* Register the volumes to the CT template, and infer coarse AC-PC estimates.
* Record results - coarse localized AC-PC will be written to a csv, along with a screenshot of the landmarks

In [ ]:
import os, re, time
import ScreenCapture
import pandas as pd
import numpy as np

In [ ]:
import Elastix
import vtk
import slicer
import sys
import SimpleITK as sitk
from pathlib import Path

In [ ]:
def isotropic_resample(input_vol_name):
    """Resample input volumes to be of 1mm^3 resolution using bspline interpolation"""
    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")
    outputVolumeNode.SetName(f'{input_vol_name} resampled')
    input_vol_node = slicer.util.getNode(f'{input_vol_name}')
    parameters = {"outputPixelSpacing":"1.0,1.0,1.0", 
                  "InputVolume":input_vol_node,
                  "interpolationType":'bspline',
                  "OutputVolume":slicer.util.getNode(f'{input_vol_name} resampled')}
    slicer.cli.run(slicer.modules.resamplescalarvolume, None, parameters, wait_for_completion=True)

In [ ]:
def crop_volumes(input_vol_name):
    """Crop the volume - this resets the direction cosine to be identity. For scans that have multiple reformats, 
    this ensures that their slices are in perfect orthogonal correspondence"""
    volumeNode = slicer.util.getNode(input_vol_name)
    roiNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsROINode")

    crop_module = slicer.vtkMRMLCropVolumeParametersNode()
    slicer.mrmlScene.AddNode(crop_module)

    crop_module.SetInputVolumeNodeID(volumeNode.GetID())
    crop_module.SetROINodeID(roiNode.GetID())

    slicer.modules.cropvolume.logic().FitROIToInputVolume(crop_module)
    slicer.modules.cropvolume.logic().Apply(crop_module)

In [ ]:
def remove_bed(input_vol_name):
    """Remove the CT bed from the image"""
    volumeNode = slicer.util.getNode(input_vol_name)
    
    threshold = -44

    # Create segmentation
    segmentationNode = slicer.vtkMRMLSegmentationNode()
    slicer.mrmlScene.AddNode(segmentationNode)
    segmentationNode.CreateDefaultDisplayNodes() #Only needed for display
    segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(volumeNode)

    # Create segment editor to get access to effects
    slicer.app.processEvents()
    segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()    
    segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
    segmentEditorNode = slicer.vtkMRMLSegmentEditorNode()
    slicer.mrmlScene.AddNode(segmentEditorNode)
    segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)
    segmentEditorWidget.setSegmentationNode(segmentationNode)
    segmentEditorWidget.setMasterVolumeNode(volumeNode)

    # Create object of interest segment by thresholding
    slicer.app.processEvents()
    volumeScalarRange = volumeNode.GetImageData().GetScalarRange()
    objectSegmentID = segmentationNode.GetSegmentation().AddEmptySegment()
    segmentEditorNode.SetSelectedSegmentID(objectSegmentID)
    segmentEditorWidget.setActiveEffectByName("Threshold")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("MinimumThreshold",str(threshold))
    effect.setParameter("MaximumThreshold",str(volumeScalarRange[1]))
    effect.self().onApply()

    # Find largest object, remove all other regions from the segment
    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName("Islands")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameterDefault("Operation", "KEEP_LARGEST_ISLAND")
    effect.self().onApply()

    # Fill holes in the segment to create a solid region of interest
    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName("Wrap Solidify")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("region", "outerSurface")
    effect.setParameter("outputType", "segment")
    effect.setParameter("remeshOversampling", 0.3)  # speed up solidification by lowering resolution
    effect.self().onApply()

    # Blank out the volume outside the object segment
    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName('Mask volume')
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter('FillValue', -1000)
    effect.setParameter('Operation', 'FILL_OUTSIDE')
    effect.self().onApply()

    # Remove temporary nodes and widget
    segmentEditorWidget = None
    slicer.mrmlScene.RemoveNode(segmentEditorNode)
    slicer.mrmlScene.RemoveNode(segmentationNode)

    # Show masked volume
    maskedVolume = slicer.mrmlScene.GetFirstNodeByName(input_vol_name+" masked")
    maskedVolume.SetName(f'{input_vol_name.split(" ")[0]} bed removed')
    slicer.util.setSliceViewerLayers(background=maskedVolume)
    

In [ ]:
def add_gauss_blur(input_vol_name, sigma):
    """Smooth input before skull stripping, based on given sigma (standard deviation of the Gaussian kernel used for 
    smoothing)"""
    output_vol_name = input_vol_name + f'_GaussBlurred'
    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")
    outputVolumeNode.SetName(output_vol_name)
    filterModule = slicer.modules.gaussianblurimagefilter
    parameters = {}
    parameters['inputVolume'] =  slicer.util.getNode(input_vol_name)
    parameters['outputVolume'] = slicer.util.getNode(output_vol_name)
    parameters['sigma'] = sigma
    cliNode = slicer.cli.runSync(filterModule,None,parameters)
    if cliNode.GetStatus() & cliNode.ErrorsMask:
        errorText = cliNode.GetErrorText()
        slicer.mrmlScene.RemoveNode(cliNode)
        raise ValueError("CLI execution failed: " + errorText)
    slicer.mrmlScene.RemoveNode(cliNode)    

In [ ]:
def skull_strip(input_vol_name, kernel_size, normal = True):
    """Perform skull stripping based on the given kernel size"""
    masterVolumeNode = slicer.util.getNode(f'{input_vol_name}')
    
    if normal: 
        min_threshold, max_threshold = 0, 100
    else:
        #some scans that have stray parts of the bed/pillow remaining following bed removal require a 
        #larger lower threshold of 1HU as compared to the "normal" ones which work with 0 HU. 
        min_threshold, max_threshold = 1, 100
    segmentationNode = slicer.vtkMRMLSegmentationNode()
    slicer.mrmlScene.AddNode(segmentationNode)
    segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(masterVolumeNode)

    # Create segment editor to get access to effects
    slicer.app.processEvents()
    segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()

    # To show segment editor widget (useful for debugging): segmentEditorWidget.show()
    segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
    segmentEditorNode = slicer.vtkMRMLSegmentEditorNode()
    slicer.mrmlScene.AddNode(segmentEditorNode)
    segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)
    segmentEditorWidget.setSegmentationNode(segmentationNode)
    segmentEditorWidget.setMasterVolumeNode(masterVolumeNode)

    # Create object of interest segment by thresholding
    slicer.app.processEvents()
    objectSegmentID = segmentationNode.GetSegmentation().AddEmptySegment()
    segmentEditorNode.SetSelectedSegmentID(objectSegmentID)
    segmentEditorWidget.setActiveEffectByName("Threshold")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("MinimumThreshold",str(min_threshold))
    effect.setParameter("MaximumThreshold",str(max_threshold))
    effect.self().onApply()

    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName("Margin")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("MarginSizeMm", -1*kernel_size)
    effect.self().onApply()

    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName("Islands")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameterDefault("Operation", "KEEP_LARGEST_ISLAND")
    effect.self().onApply()

    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName("Margin")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("MarginSizeMm", kernel_size)
    effect.self().onApply()

    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName('Smoothing')
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter('SmoothingMethod', 'MORPHOLOGICAL_CLOSING')
    effect.setParameter('KernelSizeMm', 1)
    effect.self().onApply()

    maskable_node = slicer.util.getNode(f'{input_vol_name}')

    segmentEditorWidget.setMasterVolumeNode(maskable_node)
    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName('Mask volume')
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter('FillValue', -1000)
    effect.setParameter('Operation', 'FILL_OUTSIDE')
    effect.self().onApply()

     # Remove temporary nodes and widget
    segmentEditorWidget = None
    slicer.mrmlScene.RemoveNode(segmentEditorNode)
    slicer.mrmlScene.RemoveNode(segmentationNode)

    volume_nodes = [x.GetName() for x in slicer.util.getNodesByClass('vtkMRMLScalarVolumeNode')]
    masked_node_name = [x for x in volume_nodes if re.match('.*mask.*',x)][0]
    node = slicer.util.getNode(masked_node_name)
    node.SetName(f'{input_vol_name.split(" ")[0]} brain')

In [ ]:
def ss_check(input_vol_name):
    """Check if skull-stripping worked or returned an all-zero or near-zero volume in the 0 - 80 HU range"""
    arr = slicer.util.arrayFromVolume(slicer.util.getNode(f'{input_vol_name}'))
    arr[(arr<0) | (arr > 80)] = 0
    arr_sum = np.sum(arr)
    ss_check_fail = arr_sum <=10000000
    
    return ss_check_fail

In [ ]:
def clone_node(node_name):
    """Make a clone of a given node"""
    shNode = slicer.vtkMRMLSubjectHierarchyNode.GetSubjectHierarchyNode(slicer.mrmlScene)
    nodeToClone = slicer.util.getNode(node_name)
    itemIDToClone = shNode.GetItemByDataNode(nodeToClone)
    clonedItemID = slicer.modules.subjecthierarchy.logic().CloneSubjectHierarchyItem(shNode, itemIDToClone)
    clonedNode = shNode.GetItemDataNode(clonedItemID)

In [ ]:
def apply_transform(input_node_name,trans_name,output_name):
    """Apply a transform by just 'hardening' it to the volume"""
    input_node = slicer.util.getNode(input_node_name)
    transform_node = slicer.util.getNode(trans_name)
    input_node.SetAndObserveTransformNodeID(transform_node.GetID())
    input_node.HardenTransform()
    input_node.SetName(output_name)

In [ ]:
def align_centers(input_vol_name, ref_vol_name):
    """This function aligns the physical center of a volume to that of a reference volume"""
    ref_node = slicer.util.getNode(ref_vol_name)
    vol_node = slicer.util.getNode(input_vol_name)
    
    ref_center = ref_node.GetImageData().GetCenter()       
    vol_center = vol_node.GetImageData().GetCenter()

    ref_ijk2ras = vtk.vtkMatrix4x4()
    ref_node.GetIJKToRASMatrix(ref_ijk2ras)
    ref_ph_center = ref_ijk2ras.MultiplyPoint(list(ref_center)+[1])
    
    vol_ijk2ras = vtk.vtkMatrix4x4()
    vol_node.GetIJKToRASMatrix(vol_ijk2ras)
    vol_ph_center = vol_ijk2ras.MultiplyPoint(list(vol_center)+[1])

    translation_vec = [(x-y) for x,y in zip(ref_ph_center, vol_ph_center)]

    slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLinearTransformNode",
                                       f"CenterAlignTransform_{input_vol_name}2{ref_vol_name}")

    center_align_transform = vtk.vtkMatrix4x4()
    center_align_transform.SetElement(0,0,1)
    center_align_transform.SetElement(0,1,0)
    center_align_transform.SetElement(0,2,0)
    center_align_transform.SetElement(0,3,translation_vec[0])

    center_align_transform.SetElement(1,0,0)
    center_align_transform.SetElement(1,1,1)
    center_align_transform.SetElement(1,2,0)
    center_align_transform.SetElement(1,3,translation_vec[1])

    center_align_transform.SetElement(2,0,0)
    center_align_transform.SetElement(2,1,0)
    center_align_transform.SetElement(2,2,1)
    center_align_transform.SetElement(2,3,translation_vec[2])

    center_align_transform.SetElement(3,0,0)
    center_align_transform.SetElement(3,1,0)
    center_align_transform.SetElement(3,2,0)
    center_align_transform.SetElement(3,3,1)

    t_node = slicer.util.getNode(f"CenterAlignTransform_{input_vol_name}2{ref_vol_name}")
    t_node.SetMatrixTransformToParent(center_align_transform)

In [ ]:
def elastix_register(input_vol_name, ref_vol_name, rigid):
    """Registers a given volume to a specified template"""
    output_vol_name = input_vol_name + '_ElastixRegistered'
    output_transform_name = input_vol_name + '_ElastixRegistrationTransform'
    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")
    outputVolumeNode.SetName(output_vol_name)
    outputTransformNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLTransformNode")
    outputTransformNode.SetName(output_transform_name)
    
    try:
        
        elastixLogic = Elastix.ElastixLogic()
        if rigid == 1:
            parameterFilenames = elastixLogic.getRegistrationPresets()[1][Elastix.RegistrationPresets_ParameterFilenames]
        else:
            parameterFilenames = elastixLogic.getRegistrationPresets()[0][Elastix.RegistrationPresets_ParameterFilenames]
        useelastix = True
    except Exception as e:
        print(e)
        useelastix = False

    elastixLogic.registerVolumes(slicer.util.getNode(ref_vol_name), slicer.util.getNode(input_vol_name),
                                parameterFilenames = parameterFilenames,
                                outputTransformNode = outputTransformNode,
                                outputVolumeNode = outputVolumeNode
                                )

In [ ]:
def zoomSlice(factor, sliceNode=None):
    """Zoom slice nodes by factor.
    factor: magnitude of zoom
    sliceNode:"R";"Y";"G":"None"
    sliceNodes: list of slice nodes to change, None means all.
    """
    layoutManager = slicer.app.layoutManager()    
    if sliceNode==None:
        sliceNodes = slicer.util.getNodes('vtkMRMLSliceNode*')
    else:      
        sliceNodes = {"R": "vtkMRMLSliceNodeRed", "Y": "vtkMRMLSliceNodeYellow", "G": "vtkMRMLSliceNodeGreen"}
        sliceNodes = slicer.util.getNodes(sliceNodes[sliceNode])   
        
    for sliceNode in list(sliceNodes.values()):
        sliceWidget = layoutManager.sliceWidget(sliceNode.GetLayoutName())
        sliceWidget.sliceLogic().FitSliceToAll()        
        newFOVx = sliceNode.GetFieldOfView()[0] / factor
        newFOVy = sliceNode.GetFieldOfView()[1] / factor
        newFOVz = sliceNode.GetFieldOfView()[2]
        sliceNode.SetFieldOfView(newFOVx, newFOVy, newFOVz)
        sliceNode.UpdateMatrices()

In [ ]:
def set_visibility(view_vol_map, gt_line_name = None, coarse_line_name = None):
    
    """Function to set select volumes to be visible in desired views"""
    
    #This grabs all items and turns their visibility off
    vols = [x.GetName() for x in slicer.util.getNodesByClass('vtkMRMLScalarVolumeNode')]
    trans = [x.GetName() for x in slicer.util.getNodesByClass('vtkMRMLTransformNode')]
    others = [x.GetName() for x in slicer.util.getNodesByClass('vtkMRMLMarkupsFiducialNode')] + \
             [x.GetName() for x in slicer.util.getNodesByClass('vtkMRMLMarkupsLineNode')] + \
             [x.GetName() for x in slicer.util.getNodesByClass('vtkMRMLMarkupsAngleNode')] + \
             [x.GetName() for x in slicer.util.getNodesByClass('vtkMRMLAnnotationROINode')] + \
             [x.GetName() for x in slicer.util.getNodesByClass('vtkMRMLMarkupsROINode')]

    turn_off_nodes = vols + trans + others
    shNode = slicer.vtkMRMLSubjectHierarchyNode.GetSubjectHierarchyNode(slicer.mrmlScene)
    for node in turn_off_nodes:
        shNode.SetItemDisplayVisibility(shNode.GetItemByName(node), 0)
    slicer.util.setSliceViewerLayers(background=None, foreground=None, label=None)
    #Displays volumes in the desired view. Typically, this would be the axial volume in the red view, coronal volume in 
    #the green view and sagittal volume in the yellow view
    for view, vol in view_vol_map.items():
        slicer.app.layoutManager().sliceWidget(f'{view}').sliceLogic().GetSliceCompositeNode().SetBackgroundVolumeID(
            slicer.util.getNode(f'{vol}').GetID())
    
    #if the ground-truth AC-PC is available, this visualizes it
    if gt_line_name:
        lineNode = slicer.util.getNode(gt_line_name)
        lineNode = lineNode.GetDisplayNode()
        lineNode .SetGlyphScale(1)
        lineNode.SetVisibility(True) # Show all points
    #if coarse localized AC-PC is available, this visualizes it
    if coarse_line_name:
        lineNode = slicer.util.getNode(coarse_line_name)
        lineNode = lineNode.GetDisplayNode()
        lineNode .SetGlyphScale(1)
        lineNode.SetVisibility(True) 
        
    slicer.app.layoutManager().resetSliceViews()

In [ ]:
def capture_screenshot_ss(scan_id, screenshot_path): 
    """Captures screenshot of the screen on 3D-Slicer"""
    if not os.path.exists(screenshot_path):
        os.makedirs(screenshot_path)
    
    screenCaptureLogic = ScreenCapture.ScreenCaptureLogic()
    screenCaptureLogic.showViewControllers(False)
    screenCaptureLogic.captureImageFromView(None, os.path.join(screenshot_path, f"{scan_id}.png")) 
    screenCaptureLogic.showViewControllers(True)

In [ ]:
def window_brain(input_node, w, l):
    """Windows a volume to lie between 0 - 80 HU for optimal visualization of brain tissue"""
    disp_node = input_node.GetDisplayNode()
    disp_node.AutoWindowLevelOff()
    disp_node.SetWindowLevel(w,l)

In [ ]:
def get_fiducial_measures(scan_id, line_name):
    """Grabs the end-points/control points of a line markup (AC-PC line)"""
    acpc_acpc = slicer.util.getNode(line_name) 
    ac_acpc_vector = acpc_acpc.GetNthControlPointPositionVector(0)
    ac_mark_lps_a = [-ac_acpc_vector.GetX(),-ac_acpc_vector.GetY(),ac_acpc_vector.GetZ()]
    pc_acpc_vector = acpc_acpc.GetNthControlPointPositionVector(1)
    pc_mark_lps_a = [-pc_acpc_vector.GetX(),-pc_acpc_vector.GetY(),pc_acpc_vector.GetZ()]
    
    
    return pd.DataFrame({'scan_id':[scan_id],'pc':[pc_mark_lps_a],'ac':[ac_mark_lps_a]})

In [ ]:
def add_acpc_true(acpc_true_df, scan_id):
    """Adds reference-standard or ground-truth AC-PC annotations"""
    ac_x, ac_y, ac_z = acpc_true_df[acpc_true_df['scan_id'] == scan_id]['ac'].values[0]
    pc_x, pc_y, pc_z = acpc_true_df[acpc_true_df['scan_id'] == scan_id]['pc'].values[0]

    acpc_true_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsLineNode")
    acpc_true_node.SetName('ACPC_True')

    true_ac = vtk.vtkVector3d()
    true_ac.SetX(-ac_x)
    true_ac.SetY(-ac_y)
    true_ac.SetZ(ac_z)
    true_pc = vtk.vtkVector3d()
    true_pc.SetX(-pc_x)
    true_pc.SetY(-pc_y)
    true_pc.SetZ(pc_z)

    acpc_true_node.AddControlPoint(true_ac)
    acpc_true_node.AddControlPoint(true_pc)
    
    displayNode = acpc_true_node.GetDisplayNode()
    green_color = (0.0, 1.0, 0.0)
    
    displayNode.SetColor(green_color)
    displayNode.SetSelectedColor(green_color)
    
    displayNode.SetPropertiesLabelVisibility(False)
    displayNode.SetGlyphScale(0.5)
    

In [ ]:
def process_scan(scan_id, data_path, template_path, acpc_true_df):
    """Defines the whole pipeline for image preprocessing and obtaining the coarse localized AC-PC landmarks"""
    
    slicer.mrmlScene.Clear(0)

    #Load volume
    scan_folder = data_path / str(scan_id).lstrip(os.sep)
    nifti_files = list(scan_folder.glob("*.nii"))
    
    if not nifti_files:
        sys.exit(f"No nifti files found for {scan_id} in {scan_folder}")
    
    reader = sitk.ImageFileReader()
    #grab the axial volume
    axial_img_file = None
    for file_ in nifti_files:
        reader.SetFileName(str(file_))
        reader.ReadImageInformation() 
        direction = np.array(reader.GetDirection()) 
    
        if (np.abs(np.abs(direction) - np.eye(3).flatten()) <= 0.3).all():
            axial_img_file = file_
            break
                             
    if axial_img_file:
        print(f"Processing axial volume: {axial_img_file.name}")
    else:
        sys.exit("No axial volume found")
                             
    slicer.util.loadVolume(axial_img_file, properties={'name': 'Axial'})

    #isotropic resample
    isotropic_resample('Axial')

    #crop the volume
    crop_volumes('Axial resampled')

    #remove bed
    remove_bed('Axial resampled cropped')

    ##skull stripping -----------------------   
    #gaussian blur for skull stripping
    add_gauss_blur('Axial bed removed',sigma = 2)

    #remove skull and check validity
    skull_strip(f'Axial bed removed', kernel_size = 3)

    ss_check_fail = ss_check(f'Axial brain')

    if ss_check_fail: 
        print('Skull stripping failed, retrying..')   
        slicer.mrmlScene.RemoveNode(slicer.util.getNode(f'Axial brain'))             
        skull_strip(f'Axial bed removed', kernel_size, normal = False)
        ss_check_fail = ss_check(f'Axial brain')[0]

    if ss_check_fail:     
        sys.exit(f'Check skull stripping {scan_id}')
    else:
        print(f'Skull stripping okay {scan_id}')

    window_brain(slicer.util.getNode(f'Axial brain'),80,40)

    #load template and identify AC-PC on its CIL
    slicer.util.loadVolume(template_path / 'miplab-ncct_sym_brain.nii.gz', properties = {'name': 'Template_0'})
    template_transform = slicer.util.loadTransform(str((template_path / 'miplab_to_mni_sym_warp.nii.gz').resolve()))

    template_transform.Inverse()
    template_transform.SetName('Inverse Template_MNI')
    apply_transform('Template_0','Inverse Template_MNI',
                    'Template')

    acpc_template_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsLineNode")
    acpc_template_node.SetName('ACPC_Template')

    template_ac = vtk.vtkVector3d()
    template_ac.SetX(0.0)
    template_ac.SetY(2)
    template_ac.SetZ(-2.0)
    template_pc = vtk.vtkVector3d()
    template_pc.SetX(0.0)
    template_pc.SetY(-25.0)
    template_pc.SetZ(-2.0)

    acpc_template_node.AddControlPoint(template_ac)
    acpc_template_node.AddControlPoint(template_pc)

    #center align volumes to template
    align_centers(f'Axial brain', 'Template') #this creates a transform 
    clone_node(f'Axial brain')
    apply_transform(f'Axial brain Copy',
                    f'CenterAlignTransform_Axial brain2Template',
                    f'Axial brain center aligned')

    #elastix register center aligned volumes to template
    input_vol_name = f'Axial brain center aligned'
    output_vol_name, output_transform_name = input_vol_name + '_ElastixRegistered', input_vol_name + '_ElastixRegistrationTransform'
    ref_vol_name = 'Template'
    elastix_register(input_vol_name, ref_vol_name, rigid = 0)



    #invert known AC-PC landmarks on the template's CIL to center aligned img space
    clone_node('ACPC_Template')
    clone_node(f'Axial brain center aligned_ElastixRegistrationTransform')
    el_transform_ax = slicer.util.getNode(f'Axial brain center aligned_ElastixRegistrationTransform')
    el_transform_ax.Inverse()
    el_transform_ax.SetName(f'Inverse Elastix Axial')
    apply_transform('ACPC_Template Copy',f'Inverse Elastix Axial',
                f'Axial ACPC center aligned')


    #invert inferred AC-PC landmarks from center aligned image space to native space
    clone_node(f'CenterAlignTransform_Axial brain2Template')
    clone_node(f'Axial ACPC center aligned')
    ca_transform_ax = slicer.util.getNode(f'CenterAlignTransform_Axial brain2Template Copy')
    ca_transform_ax.Inverse()
    ca_transform_ax.SetName(f'Inverse CA Axial')
    apply_transform(f'Axial ACPC center aligned Copy',f'Inverse CA Axial',
                    f'Axial brain Coarse ACPC')
    
    #write coarse localized landmark into a csv
    acpc_coarse = get_fiducial_measures(scan_id, 'Axial brain Coarse ACPC')
    acpc_coarse.to_csv(coarse_acpc_write_path,mode='a', header=not os.path.exists(coarse_acpc_write_path))

    displayNode = slicer.util.getNode('Axial brain Coarse ACPC').GetDisplayNode()
    displayNode.SetPropertiesLabelVisibility(False)
    displayNode.SetGlyphScale(0.5)
    
    displayNode.SetSliceProjection(True)
    displayNode.SetSliceProjectionOpacity(1.0)
    displayNode.SetSliceProjectionUseFiducialColor(True)
    
    #grab the true landmarks and take a screenshot of the coarse localized landmarks
    add_acpc_true(acpc_true_df, scan_id)
    
    #setting axial reformat in all views. You can choose to modify this to the typical mapping of 
    #axial to red, coronal to green, and sagittal to yellow if you have all reformats
    view_vol_map = {'Red':'Axial brain','Green':'Axial brain','Yellow':'Axial brain'}
    set_visibility(view_vol_map, gt_line_name = 'ACPC_True', coarse_line_name = 'Axial brain Coarse ACPC')
    
    #adjust slices to show the true landmark
    zoomSlice(2, sliceNode=None)

    z_level = acpc_true_df[acpc_true_df['scan_id'] == scan_id]['ac'].values[0][2] #CoarseReg
    x_level = acpc_true_df[acpc_true_df['scan_id'] == scan_id]['ac'].values[0][0]
    y_level = acpc_true_df[acpc_true_df['scan_id'] == scan_id]['ac'].values[0][1]

    redSliceNode = slicer.app.layoutManager().sliceWidget('Red').sliceLogic().GetSliceNode()
    redSliceNode.SetSliceOffset(z_level)

    yellowSliceNode = slicer.app.layoutManager().sliceWidget('Yellow').sliceLogic().GetSliceNode()
    yellowSliceNode.SetSliceOffset(x_level)

    greenSliceNode = slicer.app.layoutManager().sliceWidget('Green').sliceLogic().GetSliceNode()
    greenSliceNode.SetSliceOffset(-y_level)
    
    #capture screenshot of the coarse (pink) and true (green) AC-PC landmarks on the axial view
    capture_screenshot_ss(scan_id, screenshot_path)
    
    #save the skull-stripped axial brain volume for fine localization using the 3D-UNet
    if not os.path.exists(brain_vols_path / scan_id):
        os.makedirs(brain_vols_path / scan_id)
    slicer.util.saveNode(slicer.util.getNode('Axial brain'), 
                              str((brain_vols_path / scan_id / 'Axial brain.nii').resolve()))
    
    
    print(f'Completed processing for scan {scan_id}')
    print('----------------------------------------')

In [ ]:
#set up data read and write paths
root = Path().resolve()
data_path = root / "raw_data"
template_path = root / "ct_template"
gt_ann_path =  root / "acpc_annotations/acpc_gt.csv"
coarse_acpc_write_path = root / "acpc_annotations/acpc_coarse.csv"
screenshot_path = root / "acpc_annotations/acpc_screenshots"
brain_vols_path = root / "brain_vols"

In [ ]:
#read in true AC-PC coordinates for all scans (this would be your reference standard)
acpc_true_df = pd.read_csv(gt_ann_path)

#formatting the coordinates as a list of floats assuming that it'd be stored as a string in the format '[x, y, z]' or 
#'(x,y,z)'
acpc_true_df['ac'] = acpc_true_df['ac'].apply(lambda x: [np.float64(y.strip("[]()")) for y in x.split(",")])
acpc_true_df['pc'] = acpc_true_df['pc'].apply(lambda x: [np.float64(y.strip("[]()")) for y in x.split(",")])

In [ ]:
scan_ids = os.listdir(data_path)

In [ ]:
scan_ids

In [ ]:
slicer.app.pauseRender()
try:
    for i in range(len(scan_ids)):
        scan_id = scan_ids[i]
        process_scan(scan_id, data_path, template_path, acpc_true_df)
finally:
    slicer.app.resumeRender()
    print(f"Batch processing complete")